In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import plotly.express as px

# Temperature index models

## Acknowledgment

by Thomas Gölles 2024

## Naive first approach

In [ ]:
degree_day_factor = 8.0  # mm/day/degree celsius
temperature_plus = 3.0  # degree celsius
delta_time = 4  # day

melt = degree_day_factor * temperature_plus * delta_time
print(f"melt of snow for {delta_time} is {melt} mm")

Now lets look at the cumulative melt over 10 days

In [ ]:
n = 11
days = range(1, n)
melt = 0

for day in days:
    melt = melt + degree_day_factor * temperature_plus * delta_time
    print(f"cummulative melt of snow for {day} is {melt:.2f} mm")

ok a bit nicer, but the Temperature is still constant. Now use a random tempearature list form -5 to +10

In [ ]:
n = 11
days = range(1, n)

np.random.seed(42)
temperatures = np.random.uniform(-5, 10, len(days))
melt = 0

for day in days:
    temperature_plus = temperatures[day - 1]
    melt = melt + degree_day_factor * temperature_plus * delta_time
    print(
        f"cummulative melt of snow for {day} with {temperature_plus:.2f}°C is {melt:.2f} mm"
    )

In [ ]:
n = 11
days = range(1, n)

np.random.seed(42)
temperatures = np.random.uniform(-5, 10, len(days))
melt = 0

for day in days:
    current_temperature = temperatures[day - 1]
    temperature_plus = np.maximum(current_temperature, 0)
    melt = melt + degree_day_factor * temperature_plus * delta_time
    print(
        f"cummulative melt of snow for {day} with {current_temperature:.2f}°C is {melt:.2f} mm"
    )

## Better use of Python

If you often reuse a culation but it in a function.
A function starts with "def" then the function name you want to have followed by the input argumetnts in ().

Here I also used so called typehints like degree_day_factor: float. Which means that we expect a floating point number as the degree day factor. In Python this optional but helps a lot in understanding what is expected.

Here I also give a default value of the threshold_temp. This means that if no third argument is given the standard value of 0.0 is used.

In [ ]:
def degree_day_model(
    temperatures: np.array, degree_day_factor: float, threshold_temp: float = 0.0
) -> np.array:
    """
    Estimate daily melt amount using the Degree-Day Model.

    Parameters:
    temperatures (numpy.array): Array of daily mean air temperatures (°C).
    degree_day_factor (float): Degree-day factor (mm °C^-1 day^-1).
    threshold_temp (float): Threshold temperature above which melting occurs (°C). Default is 0.0°C.

    Returns:
    numpy.array: Array of daily melt amounts (mm).
    """
    # Calculate positive temperature differences (temperatures above the threshold)
    positive_temperatures = np.maximum(temperatures - threshold_temp, 0)

    # Calculate daily melt amounts
    daily_melt = positive_temperatures * degree_day_factor

    return daily_melt

In [ ]:
temperatures = np.array([-3.2, -1.1, 0.0, 2.2, 5.1, 3.0, 0.0, -2.0])

In [ ]:
ddf = 8.0
melt_amounts = degree_day_model(temperatures, degree_day_factor=ddf)
melt_amounts

note here we used the default value for threshold_temp.
So the result is an array of values correspondig to daily temperatures.
To make it nicer to work with lets make a pandas dataframe together with the temperatures

In [ ]:
from operator import index


melt = pd.DataFrame(
    melt_amounts,
    index=temperatures,
    columns=[f"melt (mm)"],
)
melt.index.name = "Temperature (°C)"
melt

In [ ]:
melt.plot()

### Exersize

* plot the curve for different degree day factors.
* compare snow and ice: 2.5 to 11.6 mm/(day K)  for snow 6.6 to 20.0 mm/(day K) for ice,
* change the function to use SI base units

What is the total amount of melt in mm over all days

In [ ]:
melt["melt (mm)"].sum()

## Spatial distributed version

In [ ]:
number_of_rows = 25
number_of_columns = 10

In [ ]:
np.random.seed(42)
temperatures = np.random.uniform(-5, 10, (number_of_rows, number_of_columns))

In [ ]:
melt_amounts_distributed = degree_day_model(temperatures, degree_day_factor=ddf)

In [ ]:
melt_amounts_distributed

In [ ]:
distribued_melt_df = pd.DataFrame(melt_amounts_distributed)
distribued_melt_df

In [ ]:
px.imshow(
    distribued_melt_df,
    origin="lower",
    labels=dict(x="Column", y="Row"),
    title="Plot without Interpolation",
)

## Now with space and time

In [ ]:
number_of_rows = 25
number_of_columns = 10
number_of_days = 10

In [ ]:
np.random.seed(42)
temperatures = np.random.uniform(
    -5, 10, (number_of_rows, number_of_columns, number_of_days)
)

In [ ]:
data_array = xr.DataArray(temperatures, dims=("row", "column", "day"))

In [ ]:
dataset = xr.Dataset({"temperature": data_array})

In [ ]:
dataset

In [ ]:
dataset["daily_melt"] = degree_day_model(dataset["temperature"], degree_day_factor)

In [ ]:
dataset

In [ ]:
import plotly.graph_objects as go

ds = dataset

In [ ]:
fig = go.Figure()

# Assuming 'row' and 'column' are your spatial dimensions and 'day' is the time dimension
for day in range(ds.dims["day"]):
    fig.add_trace(
        go.Heatmap(
            z=ds["daily_melt"].isel(day=day),
            x=ds["column"],
            y=ds["row"],
            showscale=False,
        )
    )

# Make 10th frame visible
# fig.data[9].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}, {"title": "Day: " + str(i)}],
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(active=10, currentvalue={"prefix": "Day: "}, steps=steps)]

fig.update_layout(sliders=sliders)

# Set up the layout
fig.update_layout(title="Daily Melt Animation", width=600, height=600, autosize=False)

fig.show()